<a href="https://colab.research.google.com/github/priigimenez/genai-foundations/blob/main/lab01_PII_Masking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PII Masking: anonimización de datos con LLMs

El objetivo de este laboratorio es implementar un pipeline de Data Masking automatizado utilizando modelos de lenguaje extenso (LLMs).
Se busca transformar datos sensibles en versiones seguras, listas para ser utilizadas para análisis, soporte o entrenamiento, sin comprometer la privacidad del usuario.


En este proyecto, como en todos los que se quieren llevar a cabo, el Data Governance no es una capa externa, sino el núcleo del desarrollo:
1. Privacidad por diseño (Privacy by Design): implementamos un filtro que detecta PII (Personally Identifiable Information) antes de que el dato sea procesado por otros sistemas.
2. Calidad del dato (Data Quality): evaluamos la precisión del LLM para identificar entidades (nombres, DNI, emails) con el fin de evitar "fugas" de datos.
3. Gestión de resgos (Risk Management): documentamos los límites del modelo (alucinaciones o falsos negativos) para entender hasta dónde podemos confiar en la automatización.
4. FinOps & Resource Governance: se implementa un control de errores para gestionar los Rate Limits de la API (Error 429), asegurando un uso responsable de los recursos de cómputo.

## Stack Tecnológico

- Lenguaje: Python 3.10+

- Modelo: 'gemini-flash-lite-latest'

- Plataforma: Google Cloud (Vertex AI / Google AI Studio)

- Framework: google-genai (SDK 2.0)

## Implementación técnica

## System Instruction

## Test Cases